In [1]:
import duckdb
import os
import sys

In [2]:
diretorio = '/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/'
files = os.listdir('/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas')

In [3]:
files

['eCommerce_schema.sql',
 'customer_product_ratings.csv',
 'campaign_product_subcategory.csv',
 'orders.csv',
 'supplier.csv',
 'payment_method.csv',
 'orderitem.csv',
 'customer.csv',
 'category.csv',
 'returns.csv',
 'marketing_campaigns.csv',
 'schema_picture.png',
 'product.csv',
 'subcategory.csv']

In [4]:
'eCommerce_schema.sql'.find('.csv')

-1

## Duckdb 

### read_csv

In [6]:
dic = {}
for file in files:
    if file.find('.csv') != -1:
        print(file)
        dic[file] = duckdb.read_csv(diretorio + file)

customer_product_ratings.csv
campaign_product_subcategory.csv
orders.csv
supplier.csv
payment_method.csv
orderitem.csv
customer.csv
category.csv
returns.csv
marketing_campaigns.csv
product.csv
subcategory.csv


In [7]:
dic['customer_product_ratings.csv'].limit(10)

┌──────────────────────┬─────────────┬────────────┬─────────┬──────────────────────────────────────────────┬───────────┐
│ customerproductrat…  │ customer_id │ product_id │ ratings │                    review                    │ sentiment │
│        int64         │    int64    │   int64    │ double  │                   varchar                    │  varchar  │
├──────────────────────┼─────────────┼────────────┼─────────┼──────────────────────────────────────────────┼───────────┤
│                 5760 │          83 │        206 │     2.5 │ Avoid this product, it''s a complete letdo…  │ bad       │
│                 5761 │         356 │         76 │     4.5 │ Great quality and value for money.           │ good      │
│                 5762 │          92 │        344 │     4.0 │ Very satisfied with the product. Will buy …  │ good      │
│                 5763 │         270 │        212 │     4.5 │ Outstanding product, I''m a happy customer.  │ good      │
│                 5764 │        

In [ ]:
#deu erro 
duckdb.read_csv("/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/*.csv")

### connect as default or memory

In [4]:
conn = duckdb.connect(database = ':default:', read_only = False)
conn.sql('SELECT * FROM "/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/subcategory.csv" LIMIT 10')

┌────────────────┬──────────────────┬─────────────┐
│ subcategory_id │ subcategory_name │ category_id │
│     int64      │     varchar      │    int64    │
├────────────────┼──────────────────┼─────────────┤
│              1 │ Smartphones      │           1 │
│              2 │ Laptops          │           1 │
│              3 │ Headphones       │           1 │
│              4 │ Cameras          │           1 │
│              5 │ Wearables        │           1 │
│              6 │ T-Shirts         │           2 │
│              7 │ Dresses          │           2 │
│              8 │ Jeans            │           2 │
│              9 │ Sweaters         │           2 │
│             10 │ Activewear       │           2 │
├────────────────┴──────────────────┴─────────────┤
│ 10 rows                               3 columns │
└─────────────────────────────────────────────────┘

In [ ]:
conn = duckdb.connect(database = ':default:', read_only = False)
df = conn.sql('SELECT * FROM "/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/subcategory.csv" ')
conn.sql('SELECT * FROM df LIMIT 10')

In [9]:
conn = duckdb.connect(database = ':memory:', read_only = False)
for file in files:
    if file.find('.csv') != -1:
        print(diretorio + file)
        conn.sql(f'''  SELECT * FROM "{diretorio}{file}" ''')


/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/customer_product_ratings.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/campaign_product_subcategory.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/orders.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/supplier.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/payment_method.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/orderitem.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/customer.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/category.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/returns.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/marketing_campaigns.csv
/home/rafaelfabrichimidt/Documentos/projetos/docker/postgresql/datas/product.csv
/home/rafaelfabrichimidt/Documentos/projetos/doc

### CREATE TABLE

#### CREATE SCHEMA

#### CREATE TABLE  

In [ ]:
create_schema = {}
conn = duckdb.connect(database = ':memory:', read_only = False)

for file in files:
    if file.find('.csv') != -1:
        print(file)
        schema = conn.sql(f"SELECT column_name, column_type FROM ( DESCRIBE SELECT * FROM  '{diretorio}{file}' ) ").fetchnumpy()
        column_name = schema['column_name'].tolist()
        column_type = schema['column_type'].tolist()
        variabels_type_ddl = ""
        for i in range(0,len(column_type),1):
            
            if column_type[i] == 'DOUBLE':
                    column_type[i] = 'DOUBLE PRECISION'
            
            if i != len(column_type) - 1:
                    
                texto = f"{column_name[i]} {column_type[i]}, \n "
                
            else:

                
                texto = f"{column_name[i]} {column_type[i]}"

            variabels_type_ddl = variabels_type_ddl + texto

        
            create_schema[file] = variabels_type_ddl


            

In [55]:
create_schema['campaign_product_subcategory.csv']

'campaign_product_subcategory_id BIGINT, \n campaign_id BIGINT, \n subcategory_id BIGINT, \n discount DOUBLE PRECISION'

In [58]:
database = 'postgresql_oltp'
schema = 'eCommerce'

create_table = {}
for file in create_schema:
    
    SQL_DDL = f"CREATE TABLE {database}.{schema}.{file.split('.')[0]}  ( \n {create_schema[file]}); "
    create_table[file] = SQL_DDL

    with open('DDL/' + file.split('.')[0] + '.sql','w') as f:
        f.write(create_table[file])
        f.close()

In [57]:
print(create_table['customer_product_ratings.csv'])

CREATE TABLE postgres.eCommerce.customer_product_ratings  ( 
 customerproductrating_id BIGINT, 
 customer_id BIGINT, 
 product_id BIGINT, 
 ratings DOUBLE PRECISION, 
 review VARCHAR, 
 sentiment VARCHAR); 


### INSERT TABLE 

In [5]:
conexao_postgres = conn.execute("ATTACH 'dbname=postgresql_oltp user=admin host=localhost, password=12345' AS db (TYPE POSTGRES);")

In [62]:
conexao_postgres

In [6]:
for file in files:
    if file.find('.csv') != -1:
        print(file)
        conexao_postgres.sql(f'''
                    
                    INSERT INTO
                    db.eCommerce.{file.split('.')[0]}
                    SELECT * FROM '{diretorio}{file}'
        
        ''')

customer_product_ratings.csv
campaign_product_subcategory.csv
orders.csv
supplier.csv
payment_method.csv
orderitem.csv
customer.csv
category.csv
returns.csv
marketing_campaigns.csv
product.csv
subcategory.csv


In [70]:
conexao_postgres.sql(f''' SHOW ALL TABLES ''')


┌──────────┬───────────┬──────────────────────┬──────────────────────┬─────────────────────────────────────┬───────────┐
│ database │  schema   │         name         │     column_names     │            column_types             │ temporary │
│ varchar  │  varchar  │       varchar        │      varchar[]       │              varchar[]              │  boolean  │
├──────────┼───────────┼──────────────────────┼──────────────────────┼─────────────────────────────────────┼───────────┤
│ db       │ ecommerce │ campaign_product_s…  │ [campaign_product_…  │ [BIGINT, BIGINT, BIGINT, DOUBLE]    │ false     │
│ db       │ ecommerce │ category             │ [category_id, cate…  │ [BIGINT, VARCHAR]                   │ false     │
│ db       │ ecommerce │ customer             │ [customer_id, firs…  │ [BIGINT, VARCHAR, VARCHAR, VARCHA…  │ false     │
│ db       │ ecommerce │ customer_product_r…  │ [customerproductra…  │ [BIGINT, BIGINT, BIGINT, DOUBLE, …  │ false     │
│ db       │ ecommerce │ marketi

In [12]:
conn.sql(f"SELECT column_name, column_type FROM ( DESCRIBE SELECT * FROM  '{diretorio}{file}' ) ").fetchnumpy()

{'column_name': array(['subcategory_id', 'subcategory_name', 'category_id'], dtype=object),
 'column_type': array(['BIGINT', 'VARCHAR', 'BIGINT'], dtype=object)}